In [65]:
from kiwipiepy import Kiwi
import pandas as pd
kiwi = Kiwi()

exclusion_tags = ['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC']
f = lambda x: x in exclusion_tags


In [76]:
# load Pseudo labled data
text_list = []
label_list = []
for i in train.index:
    if "ORG" in train.loc[i,1] or "PER" in train.loc[i,1]:
        text_list.append(train.loc[i,0])
        label_list.append(train.loc[i,1])

In [77]:
new = pd.DataFrame({"text":text_list, "label":label_list})
new.to_csv('./data/new1315.tsv',sep='\t', header=None, index=False)

In [ ]:
# read Naver NER datasets
naver = pd.read_csv('./original/KoBERT-NER/data/train.tsv', sep='\t', header=None)

In [ ]:
_text_list = list(naver.loc[:,0])
_label_list = list(naver.loc[:,1])


In [ ]:
# # combine and shuffle Naver NER dataset and pseodo-labeled law data
new_text = text_list + _text_list
new_label = label_list + _label_list

import random

c = list(zip(new_text, new_label))

random.shuffle(c)

new_text, new_label = zip(*c)


In [ ]:
# save the combined data
combine = pd.DataFrame({"text":new_text, "label":new_label})
combine.to_csv('./combine.tsv',sep='\t', header=None, index=False)

In [6]:
# Easy Data Augmentation functions(random deletion, random swap)
def augment(sentence, labels, num_new_per_technique):
    words = sentence.split(" ")
    words = [word for word in words if word != ""]
    labels = labels.split(" ")
    num_words = len(words)

    augmented_sentences = []
    augmented_labels = []

    RS_ratio = 0.2
    RD_ratio = 0.2
    n_rs = max(1, int(RS_ratio*num_words))
    for _ in range(num_new_per_technique):
        a_words, a_labels = random_swap(words, labels, n_rs)
        augmented_sentences.append(' '.join(a_words))
        augmented_labels.append(' '.join(a_labels))

    for _ in range(num_new_per_technique):
        a_words, a_labels = random_deletion(words, labels, RD_ratio)
        augmented_sentences.append(' '.join(a_words))
        augmented_labels.append(' '.join(a_labels))



    c = list(zip(augmented_sentences, augmented_labels))

    random.shuffle(c)
    augmented_sentences, augmented_labels = zip(*c)
    

    return augmented_sentences, augmented_labels

In [ ]:
# Easy Data Augmentation functions(random deletion, random swap)
def random_deletion(words, labels, p):
    if len(words) == 1:
        return words, labels

    new_words = []
    new_labels = []
    for i, word in enumerate(words):
        r = random.uniform(0,1)
        if r > p:
            new_words.append(word)
            new_labels.append(labels[i])

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]], [labels[rand_int]]
    # print("RD done")
    return new_words, new_labels

def random_swap(words, labels, n):
    new_words = words.copy()
    new_labels = labels.copy()
    for _ in range(n):
        new_words, new_labels = swap_word(new_words, new_labels)
    # print("RS done")
    return new_words, new_labels

def swap_word(new_words, new_labels):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter =0

    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            break

    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    new_labels[random_idx_1], new_labels[random_idx_2] = new_labels[random_idx_2], new_labels[random_idx_1]
    return  new_words, new_labels

In [82]:
# augment data by using random deletion and random swap
# only augment pseodo-labeled law data to solve data imbalance (Naver NER data is more than 7 times of law data)

a_X = text_list
a_Y = label_list

NUM_AUG = 3
for i in range(len(a_X)):
    augmented_sentences, augmented_labels = augment(text_list[i], label_list[i], NUM_AUG)
    a_X.extend(augmented_sentences)
    a_Y.extend(augmented_labels)
    
c = list(zip(a_X, a_Y))
random.shuffle(c)
a_X, a_Y = zip(*c)

d = {'text':a_X,'label':a_Y}
aug_new = pd.DataFrame(d)

In [83]:
aug_new.to_csv('./data/augment1320_law.tsv',sep='\t', header=None, index=False)

In [ ]:
new_text = list(a_X) + _text_list
new_label = list(a_Y) + _label_list

import random

c = list(zip(new_text, new_label))

random.shuffle(c)

new_text, new_label = zip(*c)

augment = pd.DataFrame({"text":new_text, "label":new_label})
augment.to_csv('./augment.tsv',sep='\t', header=None, index=False)